In [ ]:
# install the latest version
#%pip install ultralytics

In [8]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Constants
WIDTH = 1280
HEIGHT = 924

# Load cleaned dataset
df = pd.read_csv(r"C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\cleaned_foid.csv")

# Mapping class names to class IDs
class_map = {
    'Albacore':0, 'Bigeye tuna':1, 'Black marlin':2, 'Blue marlin':3, 'Great barracuda':4, 'Human':5,
    'Indo Pacific sailfish':6, 'Long snouted lancetfish':7, 'Mahi mahi':8, 'Mola mola':9, 'No fish':10,
    'Oilfish':11, 'Opah':12, 'Pelagic stingray':13, 'Pomfret':14, 'Rainbow runner':15, 'Roudie scolar':16, 'Shark':17,
    'Shortbill spearfish':18, 'Sickle pomfret':19, 'Skipjack tuna':20, 'Snake mackerel':21, 'Striped marlin':22,
    'Swordfish':23, 'Thresher shark':24, 'Unknown':25, 'Wahoo':26, 'Yellowfin tuna':27
}

# Map labels to class IDs
df["class_id"] = df["label"].map(class_map)

# Normalize coordinates
df["x_center_norm"] = df["center_x"] / WIDTH
df["y_center_norm"] = df["center_y"] / HEIGHT
df["width_norm"] = df["width"] / WIDTH
df["height_norm"] = df["height"] / HEIGHT

# Train/val split
unique_ids = df["img_id"].unique()
train_ids, val_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

# Label writing function
def write_labels(img_ids, split):
    for img_id in img_ids:
        group = df[df["img_id"] == img_id]
        label_path = f"C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/SCRIPTS/labels/{split}/{img_id}.txt"
        os.makedirs(os.path.dirname(label_path), exist_ok=True)
        with open(label_path, "w") as f:
            for _, row in group.iterrows():
                f.write(f"{row['class_id']} {row['x_center_norm']:.6f} {row['y_center_norm']:.6f} {row['width_norm']:.6f} {row['height_norm']:.6f}\n")

# Write to train and val folders
write_labels(train_ids, "train")
write_labels(val_ids, "val")

print("Label files successfully written for training and validation sets.")


Label files successfully written for training and validation sets.


In [9]:
# Define paths 
train_path = 'C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/SCRIPTS/images/train'
val_path = r'C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\val'
#test_path = 'C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/images/test'

# Get list of class names from the class_map (in correct order)
class_names = [None] * len(class_map)
for name, idx in class_map.items():
    class_names[idx] = name

# Create dictionary for YAML
data_yaml = {
    'val': val_path.replace('\\', '/'),
    'train': train_path.replace('\\', '/'),
    'names': class_names,
    'nc': len(class_map)
}

# Write to YAML
with open('C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/SCRIPTS/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [10]:
from ultralytics import YOLO
import os

#os.chdir(r"C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS")

#pretrained model
model = YOLO("yolov8s.pt")

#data_path = os.path.abspath("data.yaml")

In [11]:
model.info()

YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [12]:
model.train(
    data=r"C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/SCRIPTS/data.yaml",
    epochs=5,
    imgsz=640,
    batch=8,
    name="fishnet_yolov8",
    device='cpu',
    workers=4,
    project=r"C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/runs",  # explicit output directory
    exist_ok=True  # overwrite existing project
)

Ultralytics 8.3.110  Python-3.11.9 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/SCRIPTS/data.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=C:/Users/Brian/.vscode/DS4002-Project3/DSProject3/runs, name=fishnet_yolov8, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

train: Scanning C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\labels\train... 1642 images, 415 backgrounds, 

train: WARNING  C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\train\94f61b32-23f0-11e9-b8a2-c31594a9d38f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1496]
train: WARNING  C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\train\94f61baa-23f0-11e9-b8a3-17add2042905.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1492]
train: WARNING  C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\train\94f61c2c-23f0-11e9-b8a4-4bc41fae21d8.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.118]
train: WARNING  C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\train\94f61eac-23f0-11e9-b8a5-4f1fad99488d.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1477]
train: WARNING  C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\images\train\94f61f38-23f0-11e9-b8a6-4fff68f4a587.jpg: ign

train: New cache created: C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\labels\train.cache
val: Fast image access  (ping: 0.40.1 ms, read: 172.966.6 MB/s, size: 138.4 KB)


val: Scanning C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\SCRIPTS\labels\val... 413 images, 1639 backgrounds, 23 c

...

                   all       2029       1241      0.235      0.502      0.146      0.107



5 epochs completed in 14.390 hours.
Optimizer stripped from C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\runs\fishnet_yolov8\weights\last.pt, 22.5MB
Optimizer stripped from C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\runs\fishnet_yolov8\weights\best.pt, 22.5MB

Validating C:\Users\Brian\.vscode\DS4002-Project3\DSProject3\runs\fishnet_yolov8\weights\best.pt...
Ultralytics 8.3.110  Python-3.11.9 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
Model summary (fused): 72 layers, 11,136,420 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 127/127 [05:5


                   all       2029       1241      0.235      0.503      0.147      0.107
              Albacore        127        135      0.122      0.415     0.0982     0.0444
           Bigeye tuna         24         24      0.134        0.5      0.146     0.0995
           Blue marlin          1          1          0          0          0          0
                 Human        368        830       0.19      0.817      0.186      0.128
Long snouted lancetfish          4          4      0.235        0.5      0.201      0.171
             Mahi mahi         65         78      0.175      0.555      0.148     0.0799
               No fish          4          4     0.0679        0.5      0.143      0.111
                 Shark          2          2          1          0          0          0
             Swordfish          6          6      0.234          1      0.366       0.33
        Yellowfin tuna        139        157      0.193      0.739      0.177      0.106
Speed: 2.1ms preproc

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  3,  5,  7,  8, 10, 17, 23, 27])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000025297F0AC10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.

In [ ]:
metrics = model.val()